# Build (AOIs)
This is the initalizing notebook for building a spatially enabled SQLite database supporting the GAIA program. The notebook identifies downloaded Keyhole Mark-up Language (KMLs) files, denoting Areas of Interest (AOIs), converts them to GeoJSON files, builds a GeoPandas GeoDataFrame, then commits this to the SQLite Database.

### Import libraries

In [1]:
# Basic stack
import os
import shutil
from glob import glob

# Web stack
import json

# Database stack
import sqlite3

# Data Science stack
import pyproj
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import utm
import pandas as pd
import geopandas as gpd
import shapely
from shapely import to_geojson
from shapely.ops import transform
from shapely.geometry import Polygon
from fiona.drvsupport import supported_drivers
import folium

# Custom stack
import sys; sys.path.append("../../")
from EE import security, search

### User defined variables

In [2]:
transfer_aois = "C:/gis/gaia/transfer/aois"
dir_kml = "C:/gis/gaia/data/kmls"
db = "C:/gis/gaia/data/databases/gaia.db"

### User defined functions

In [3]:
def select_data(c, columns):
    """ Selects all data from the `aois` table for review

        C - A cursor
        COLUMNS - Columns of interest
    """
    columns = ', '.join(columns)
    sql_string = f"SELECT {columns}, AsText(geom) FROM aois"
    c.execute(sql_string)
    return c.fetchall()

def update_aoi(c, idd, name, utm, sqkm, geom):
    """ Insert/update the `aois` SQLite Data Table with a new Area of Interest.

        C - A cursor
        AOI ID - The AOI ID
        NAME - The location name
        UTM - The UTM Zone as an EPSG code
        SQKM - Square kilometers
        GEOM - The area of interest's geometry
    """
    sql_string = (f"INSERT INTO aoi(aid, name, utm, sqkm, geom) \n"
                     f"VALUES ({idd}, \"{name}\", \"{utm}\", {sqkm}, ST_GeomFromText(\"{geom}\", 4326))")
    c.execute(sql_string)

def utm_and_area(aoi):
    """ Determines the UTM Zone, as an EPSG code, of an Area of Interest
            as well as culates the area, in square kilometers, of this
            Area of Interest.

        AOI - Area of Interest as a Shapely Polygon
    """
    minx, miny, maxx, maxy = aoi.bounds
    
    utm_crs_list = query_utm_crs_info(
        datum_name = "WGS 84",
        area_of_interest=AreaOfInterest(
            west_lon_degree = minx,
            south_lat_degree = miny,
            east_lon_degree = maxx,
            north_lat_degree = maxy,
        ),
    )

    if len(utm_crs_list) == 1:
        print("One zone, {}, was identified and will be used for calculations.".format(utm_crs_list[0].name))
    else:
        print("Multiple zones were identified. {} will be used for calculations.".format(utm_crs_list[0].name))
        alt_utms = [utm_crs.name for utm_crs in utm_crs_list][1:]
        print("\tOther options were: {}".format(alt_utms))
    
    utm_crs = CRS.from_epsg(utm_crs_list[0].code)
    proj_wgs2utm = pyproj.Transformer.from_crs(pyproj.CRS('EPSG:4326'), utm_crs, always_xy=True).transform
    sqkm = transform(proj_wgs2utm, aoi).area / 1_000_000
    return utm_crs, sqkm

def geojson_to_row(geojson):
    """ Creates a GeoDataFrame from a GeoJSON file then adds columns for
            the AOI ID, location name, UTM Zone (as an EPSG code), and
            area in square kilometers.

        Is dependent on the UTM_AND_AREA function above.

        GEOJSON - A GeoJSON file
    """
    try:
        gdf = gpd.read_file(geojson, crs='EPSG:4326')
        root_name = geojson.split('/')[-1]
        gdf['id'] = root_name.split('.')[0].split('-')[0]
        gdf['name'] = root_name.split('.')[0].split('-')[1]
        utm_crs, sqkm = utm_and_area(gdf['geometry'][0])
        gdf['utm'] = utm_crs
        gdf['sqkm'] = round(sqkm, 2)
        return gdf
    except Exception as e:
        print("Failed on {} due to: {}".format(geojson, e))
        pass

def kml_to_geojson(kml, out_dir):
    """ Converts a KML file to GeoJSON file.

        KML - KML file
        OUT DIR - Output directory
    """
    supported_drivers['KML'] = 'rw'

    kml_name = kml.split('\\')[-1]
    gdf = gpd.read_file(kml, driver='KML')
    if len(gdf["geometry"]) == 1:
        kml_shape = to_geojson(gdf['geometry'][0])

        root_name = kml_name.split('.')[0]
        geojson_file = "{}/{}.geojson".format(out_dir, root_name)
        with open(geojson_file, "w") as f:
            f.write(kml_shape)
            f.close()
        
        with open(geojson_file, "r") as f:
            aoi = json.loads(f.read())
            f.close()

        return geojson_file
            
    elif len(gdf["geometry"]) > 1:
        print("More than one geometry found in your {} KML. Passing...".format(kml_name))
        pass

### Identify, move Areas of Interest (AOIs) as KMLs

In [4]:
aoi_kmls = glob(transfer_aois + "/*.kml")
for aoi_kml in aoi_kmls:
    aoi_name = aoi_kml.split('\\')[-1]
    kml_aoi = "{}/{}".format(dir_kml, aoi_name)
    shutil.move(aoi_kml, kml_aoi)

### Provide some feedback to the user on what was moved

In [5]:
transfer_files = glob(transfer_aois + "/*.*")
kml_count = len(aoi_kmls) - len(transfer_files)
print("{} KMLs were moved. You still have {} files in your transfer aois directory".format(kml_count, len(transfer_files)))

-25 KMLs were moved. You still have 25 files in your transfer aois directory


### Convert AOI KMLs to GeoJSON files

In [6]:
aoi_kmls = glob(dir_kml + "/*.kml")
dir_geojson = "C:/gis/gaia/data/geojson"
aoi_geojsons = [kml_to_geojson(aoi_kml, dir_geojson) for aoi_kml in aoi_kmls]

More than one geometry found in your 0135-RemoteSensingSI-NWFSC-WestofSanJuans.kml KML. Passing...


### Read GeoJSONs into GeoDataFrame

In [7]:
gdf = pd.concat([geojson_to_row(aoi_geojson) for aoi_geojson in aoi_geojsons], ignore_index=True)
corrected_columns = ['id', 'name', 'utm', 'sqkm', 'geometry']
gdf = gdf[corrected_columns]
gdf = gdf.rename(columns = {'id': 'aid'})
gdf.head()

One zone, WGS 84 / UTM zone 5N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 5N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 5N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 5N, was identified and will be used for calculations.
Multiple zones were identified. WGS 84 / UTM zone 5N will be used for calculations.
	Other options were: ['WGS 84 / UTM zone 6N']
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identif

,aid,name,utm,sqkm,geometry
0,0001,Kenai,EPSG:32605,416.53,"POLYGON ((-151.59303 60.32094, -151.43153 60.6..."
1,0002,Kalgin Island,EPSG:32605,884.65,"POLYGON ((-151.73795 60.71243, -151.69676 60.5..."
2,0003,Trading Bay,EPSG:32605,529.37,"POLYGON ((-151.34361 61.01015, -151.22669 60.9..."
3,0004,Tuxedni,EPSG:32605,438.72,"POLYGON ((-152.60081 60.02992, -152.59393 60.0..."
4,0005,Upper CI,EPSG:32605,1871.28,"POLYGON ((-150.67303 60.95571, -151.00321 61.1..."


In [8]:
gdf.shape

(55, 5)

In [9]:
gdf.head(55)

,aid,name,utm,sqkm,geometry
0,0001,Kenai,EPSG:32605,416.53,"POLYGON ((-151.59303 60.32094, -151.43153 60.6..."
1,0002,Kalgin Island,EPSG:32605,884.65,"POLYGON ((-151.73795 60.71243, -151.69676 60.5..."
2,0003,Trading Bay,EPSG:32605,529.37,"POLYGON ((-151.34361 61.01015, -151.22669 60.9..."
3,0004,Tuxedni,EPSG:32605,438.72,"POLYGON ((-152.60081 60.02992, -152.59393 60.0..."
4,0005,Upper CI,EPSG:32605,1871.28,"POLYGON ((-150.67303 60.95571, -151.00321 61.1..."
5,0006,Cape Cod Bay,EPSG:32619,1347.04,"POLYGON ((-70.11938 42.10724, -70.52386 42.109..."
6,0007,Cape Cod Bay north,EPSG:32619,1291.27,"POLYGON ((-70.62919 42.10035, -70.55071 41.965..."
7,0008,North of Cape Cod,EPSG:32619,2145.21,"POLYGON ((-70.09225 42.16102, -70.02663 42.475..."
8,0009,East of Cape Cod,EPSG:32619,722.59,"POLYGON ((-69.93751 41.86854, -69.93751 41.605..."
9,0010,"East of Cape Cod, Monomoy",EPSG:32619,1396.58,"POLYGON ((-69.93951 41.55669, -69.31993 41.534..."


### Drop table
This is for demonstration purposes

In [10]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS aois''')
conn.commit()
conn.close()

### Create databse, connect, and create AOI data table

In [11]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS aoi(
        aid INTEGER PRIMARY KEY,
        name VARCHAR(50),
        requestor VARCHAR(25),
        utm VARCHAR(10),
        sqkm NUMERIC(10, 2)
    )
''')

c.execute('''SELECT AddGeometryColumn('aoi', 'geom', 4326, 'POLYGON')''')

conn.commit()
conn.close()

### Using the GeoDataFrame, insert the AOIs into the `aois` table

In [12]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

for i, row in gdf.iterrows():
    try:
        update_aoi(c, row['aid'], row['name'], row['utm'], row['sqkm'], row['geometry'])
    except Exception as e:
        print("Exception: {} was raised for AOI ID {}".format(e, row['aid']))
print("Done updating AOI table!")

conn.commit()
conn.close()

Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0001
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0002
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0003
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0004
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0005
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0006
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0007
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0008
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0009
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0010
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0011
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0012
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI ID 0013
Exception: UNIQUE constraint failed: aoi.aid was raised for AOI 

### Select newly inserted AOIs, make a GeoDataFrame for validation

In [13]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

main_columns = list(gdf.columns)[:-1]
main_columns = ', '.join(main_columns)
df = pd.read_sql_query(f"SELECT {main_columns}, AsText(geom) FROM aoi", conn)
df = df.rename(columns={'AsText(geom)': 'geometry'}, errors='raise')
df['geometry'] = shapely.wkt.loads(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry='geometry')

conn.commit()
conn.close()

gdf.head()

,aid,name,utm,sqkm,geometry
0,1,Kenai,EPSG:32605,416.53,"POLYGON ((-151.59303 60.32094, -151.43153 60.6..."
1,2,Kalgin Island,EPSG:32605,884.65,"POLYGON ((-151.73795 60.71243, -151.69676 60.5..."
2,3,Trading Bay,EPSG:32605,529.37,"POLYGON ((-151.34361 61.01015, -151.22669 60.9..."
3,4,Tuxedni,EPSG:32605,438.72,"POLYGON ((-152.60081 60.02992, -152.59393 60.0..."
4,5,Upper CI,EPSG:32605,1871.28,"POLYGON ((-150.67303 60.95571, -151.00321 61.1..."


In [14]:
# Note that the GDF shape matches that from the above
gdf.shape

(55, 5)

### Plot Areas of Interest on an Interactive Map

In [15]:
def style_function(hex_value):
    return {'color': hex_value, 'fillOpacity': 0}

# Add OpenStreetMap as a basemap
map = folium.Map()
folium.TileLayer('openstreetmap').add_to(map)

# Create a GeoJson layer from the response_geojson and add it to the map
folium.GeoJson(
    gdf['geometry'].to_json(),
    style_function = lambda x: style_function('#0000FF')
).add_to(map)

# Zoom to collected images
map.fit_bounds(map.get_bounds(), padding=(100, 100))

# Display the map
map

# End